In [46]:
import os
from dotenv import load_dotenv
load_dotenv()

from openai import OpenAI

MODEL = "provider-3/gpt-4.1-mini"

a4f_client = OpenAI(
  base_url="https://api.a4f.co/v1",
  api_key=os.getenv("OPENAI_API_KEY"),
)

In [47]:
import requests
import json

def search_gutenberg_books(search_terms: str):
    """Search for books on Project Gutenberg based on search terms."""
    search_query = " ".join(search_terms)
    url = f"https://gutendex.com/books"
    response = requests.get(url, params={"search": search_query})
    response.raise_for_status()

    simplified_results = []
    for book in response.json().get("results", []):
        simplified_results.append({
            "id": book.get("id"),
            "title": book.get("title"),
            "authors": [author.get("name") for author in book.get("authors", [])], # Extract author names
        })
    return simplified_results

tools = [
  {
    "type": "function",
    "function": {
      "name": "search_gutenberg_books",
      "description": "Search for books on Project Gutenberg based on search terms. Returns a list of books with their titles and authors.",
      "parameters": {
        "type": "object",
        "properties": {
          "search_terms": {
            "type": "string",
            "description": "The search terms to look for, e.g. 'Leo Tolstoy great gatsby'"
          }
        },
        "required": ["search_terms"]
      }
    }
  }
]

TOOL_MAPPING = {
    "search_gutenberg_books": search_gutenberg_books,
}

In [48]:
def call_llm(msgs):
    resp_completion = a4f_client.chat.completions.create(
        model=MODEL,
        tools=tools,
        messages=msgs
    )
    msgs.append(resp_completion.choices[0].message)
    return resp_completion

def get_tool_response_from_llm_response(llm_completion_response):
    if not llm_completion_response.choices[0].message.tool_calls:
        return None

    tool_call = llm_completion_response.choices[0].message.tool_calls[0]
    tool_name = tool_call.function.name
    tool_args_str = tool_call.function.arguments
    try:
        tool_args = json.loads(tool_args_str)
    except json.JSONDecodeError:
        print(f"Error decoding arguments for {tool_name}: {tool_args_str}")
        return {
            "role": "tool",
            "tool_call_id": tool_call.id,
            "name": tool_name,
            "content": json.dumps({"error": "Invalid arguments format received from LLM."})
        }

    if tool_name in TOOL_MAPPING:
        function_to_call = TOOL_MAPPING[tool_name]
        try:
            tool_result = function_to_call(**tool_args)
        except Exception as e:
            print(f"Error executing tool {tool_name}: {e}")
            tool_result = {"error": f"Failed to execute tool {tool_name}: {str(e)}"}
    else:
        print(f"Tool {tool_name} not found in mapping.")
        tool_result = {"error": f"Tool {tool_name} not recognized."}

    return {
        "role": "tool",
        "tool_call_id": tool_call.id,
        "name": tool_name,
        "content": json.dumps(tool_result)
    }

current_messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What books did James Joyce write?"}
]

MAX_ITERATIONS = 5
for i in range(MAX_ITERATIONS):
    print(f"--- Iteration: {i + 1} ---")
    llm_response_completion = call_llm(current_messages)

    if llm_response_completion.choices[0].message.tool_calls:
        print("LLM requested tool call(s).")
        tool_message = get_tool_response_from_llm_response(llm_response_completion)
        if tool_message:
            current_messages.append(tool_message)
    else:
        print("LLM finished, no tool calls requested.")
        final_answer = llm_response_completion.choices[0].message.content
        print(f"\nFinal Answer:\n{final_answer}")
        break
else:
    print("Max iterations reached without a final answer.")

--- Iteration: 1 ---
LLM requested tool call(s).


KeyboardInterrupt: 